In [ ]:
"""
Holling-Tanner Model with FUZZY Initial Conditions
=====================================================

Tests scenarios with fuzzy prey and predators
in cases WITH and WITHOUT limit cycles.

Holling-Tanner Model:
    dx/dt = r * x * (1 - x/K) - (a*x*y) / (b + x)
    dy/dt = c * y*(1-d*y/x)
Scenarios:
    1. WITH cycles + Fuzzy Prey and Predators
    2. WITHOUT cycles + Fuzzy Prey and Predators
"""


import numpy as np
import matplotlib.pyplot as plt
import warnings

# Suppress scipy warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)


import sys
import os

# Add paths to import fuzzy_systems module
sys.path.insert(0, os.getcwd())
sys.path.insert(1, os.path.abspath('../..'))

from fuzzy_systems.dynamics.fuzzy_ode import FuzzyODESolver, FuzzyNumber

In [ ]:
# ============================================================================
# HOLLING-TANNER MODEL
# ============================================================================

def holling_tanner_model(t, z, r, K, m, d, a, b):
    """
    Holling-Tanner predator-prey model.

    Arguments:
        t: Time (not used)
        z: Vector [x, y] (prey, predator)
        r: Prey growth rate
        K: Prey carrying capacity
        m: Maximum predation rate
        d: Saturation density
        a: Conversion efficiency
        b: Predator mortality rate

    Returns:
        dy/dt = [dx/dt, dy/dt]
    """
    x, y = z

    # Model equations
    dx_dt = r * x * (1 - x/K) - (m*x*y) / (d + x)
    dy_dt = a * y*(1-b*y/x)

    return np.array([dx_dt, dy_dt])


# ============================================================================
# PARAMETER SETS
# ============================================================================

# Case 1: WITHOUT cycles (stable equilibrium)
params_sem_ciclos = {
    'r': 0.2,
    'K': 100.0,
    'm': 0.03,
    'd': 20.0,
    'a': 0.02,
    'b': 0.155
}

# Case 2: WITH cycles (persistent oscillations)
params_com_ciclos = {
    'r': 0.2,
    'K': 100.0,
    'm': 0.03,
    'd': 20.0,
    'a': 0.02,
    'b': 0.12
}

In [ ]:
# ============================================================================
# SCENARIO 1: WITHOUT CYCLES + FUZZY INITIAL CONDITIONS
# ============================================================================

# Time span for simulation
t_sim = (0, 700)

# Fuzzy initial conditions: prey centered at 40 ± 5, predator at 245 ± 5
x0_fuzzy = FuzzyNumber.triangular(center=40, spread=5)
y0_fuzzy = FuzzyNumber.triangular(center=245, spread=5)

# Create solver with parameters for stable equilibrium (no cycles)
solver = FuzzyODESolver(
    ode_func=holling_tanner_model,
    t_span=t_sim,
    initial_condition=[x0_fuzzy, y0_fuzzy],
    params=params_sem_ciclos,
    n_alpha_cuts=20,
)

# Solve using Monte Carlo method with 20000 samples
sol = solver.solve(method='monte_carlo', n_samples=20000, verbose=True)
sol.plot()

In [ ]:
# ============================================================================
# SCENARIO 2: WITH CYCLES + FUZZY INITIAL CONDITIONS
# ============================================================================

# Time span for simulation
t_sim = (0, 700)

# Fuzzy initial conditions: prey centered at 40 ± 5, predator at 245 ± 5
x0_fuzzy = FuzzyNumber.triangular(center=40, spread=5)
y0_fuzzy = FuzzyNumber.triangular(center=245, spread=5)

# Create solver with parameters for persistent oscillations (with cycles)
solver = FuzzyODESolver(
    ode_func=holling_tanner_model,
    t_span=t_sim,
    initial_condition=[x0_fuzzy, y0_fuzzy],
    params=params_com_ciclos,
    n_alpha_cuts=20,
)

# Solve using Monte Carlo method with 20000 samples
sol = solver.solve(method='monte_carlo', n_samples=20000, verbose=True)
sol.plot()